In [83]:
import pandas as pd
import torch
import numpy as np
from tqdm.notebook import tqdm
import requests
import time
import json

import concurrent.futures
from openai import OpenAI

In [19]:
working_space = "/mnt/lab_data2/ryanzhao/gene_drug_graph"

In [6]:
df = pd.read_csv("/mnt/lab_data/kundaje/users/chihoim/datasets/motive_data/inputs/orf/meta.csv.gz", compression='gzip')
df

,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol,Metadata_NCBI_Gene_ID,Metadata_Taxon_ID,Metadata_Gene_Description,Metadata_Prot_Match,Metadata_Insert_Length,Metadata_pert_type
0,JCP2022_900002,ccsbBroad304_00001,ORF008415.1_TRC304.1,pLX_304,NM_001160173.3,NAT1,9,9606,N-acetyltransferase 1,100.0,870.0,trt
1,JCP2022_900003,ccsbBroad304_00002,ORF005388.1_TRC304.1,pLX_304,NM_001088.3,AANAT,15,9606,aralkylamine N-acetyltransferase,100.0,621.0,trt
2,JCP2022_900004,ccsbBroad304_00003,ORF003876.1_TRC304.1,pLX_304,NM_000663.4,ABAT,18,9606,4-aminobutyrate aminotransferase,100.0,1500.0,trt
3,JCP2022_900005,ccsbBroad304_00007,ORF004679.1_TRC304.1,pLX_304,NM_000018.4,ACADVL,37,9606,acyl-CoA dehydrogenase very long chain,100.0,1965.0,trt
4,JCP2022_900006,ccsbBroad304_00008,ORF000425.1_TRC304.1,pLX_304,NM_001095.4,ASIC1,41,9606,acid sensing ion channel subunit 1,100.0,1584.0,trt
...,...,...,...,...,...,...,...,...,...,...,...,...
15137,JCP2022_915128,ccsbBroad304_99985,ORFC00001.1_TRC304.1,pLX_304,BFP.1,BFP,BFP,CONTROL,Hahn Lab BFP,99.5,717.0,negcon
15138,JCP2022_915129,ccsbBroad304_99988,ORFC00002.1_TRC304.1,pLX_304,HcRed.1,HcRed,HcRed,CONTROL,NaN,NaN,824.0,negcon
15139,JCP2022_915130,ccsbBroad304_99991,ORFC00003.1_TRC304.1,pLX_304,promegaLuc.1,LUCIFERASE,LUCIFERASE,CONTROL,promegaLuc,100.0,1650.0,negcon
15140,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,3204.0,negcon


In [16]:
unique_metadata_symbol = df.drop_duplicates(subset="Metadata_Symbol")
unique_metadata_symbol

,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol,Metadata_NCBI_Gene_ID,Metadata_Taxon_ID,Metadata_Gene_Description,Metadata_Prot_Match,Metadata_Insert_Length,Metadata_pert_type
0,JCP2022_900002,ccsbBroad304_00001,ORF008415.1_TRC304.1,pLX_304,NM_001160173.3,NAT1,9,9606,N-acetyltransferase 1,100.0,870.0,trt
1,JCP2022_900003,ccsbBroad304_00002,ORF005388.1_TRC304.1,pLX_304,NM_001088.3,AANAT,15,9606,aralkylamine N-acetyltransferase,100.0,621.0,trt
2,JCP2022_900004,ccsbBroad304_00003,ORF003876.1_TRC304.1,pLX_304,NM_000663.4,ABAT,18,9606,4-aminobutyrate aminotransferase,100.0,1500.0,trt
3,JCP2022_900005,ccsbBroad304_00007,ORF004679.1_TRC304.1,pLX_304,NM_000018.4,ACADVL,37,9606,acyl-CoA dehydrogenase very long chain,100.0,1965.0,trt
4,JCP2022_900006,ccsbBroad304_00008,ORF000425.1_TRC304.1,pLX_304,NM_001095.4,ASIC1,41,9606,acid sensing ion channel subunit 1,100.0,1584.0,trt
...,...,...,...,...,...,...,...,...,...,...,...,...
15137,JCP2022_915128,ccsbBroad304_99985,ORFC00001.1_TRC304.1,pLX_304,BFP.1,BFP,BFP,CONTROL,Hahn Lab BFP,99.5,717.0,negcon
15138,JCP2022_915129,ccsbBroad304_99988,ORFC00002.1_TRC304.1,pLX_304,HcRed.1,HcRed,HcRed,CONTROL,NaN,NaN,824.0,negcon
15139,JCP2022_915130,ccsbBroad304_99991,ORFC00003.1_TRC304.1,pLX_304,promegaLuc.1,LUCIFERASE,LUCIFERASE,CONTROL,promegaLuc,100.0,1650.0,negcon
15140,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,3204.0,negcon


In [58]:
def _get_gene_id_entrez(gene_symbol: str):
    '''
    Get the Entrez ID for a gene symbol. If no match found, returns None
    e.g. 1017 (CDK2)
    '''
    api_call = f'https://mygene.info/v3/query?species=human&q=symbol:{gene_symbol}'
    response = requests.get(api_call)
    response_json = response.json()

    if len(response_json["hits"]) == 0:
        return None
    else:
        for hit in response_json["hits"]:
            if "entrezgene" in hit:
                return hit["entrezgene"]
        return None

In [25]:
str("hi") == "hi"

True

In [ ]:
error_log = f"{working_space}/summary_api2.err"
summary_file = f"{working_space}/summary_data2.tsv"
# first run failed on step 5028

api_base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gene&retmode=json&id="

count = 0
with open(error_log, "w") as f_err, open(summary_file, "w") as f_out:
    for i, row in tqdm(unique_metadata_symbol.iterrows(), total=len(unique_metadata_symbol)):
        count += 1
        if count < 5020:
            continue
        ncbi_id_str = row["Metadata_NCBI_Gene_ID"]
        symb = row["Metadata_Symbol"] # for logging purposes
        try:
            ncbi_id = int(ncbi_id_str)
            ncbi_id = str(ncbi_id)
        except:
            ncbi_id = _get_gene_id_entrez(row["Metadata_Symbol"])
            if ncbi_id is None:
                print("Error occured in finding stuff")
                f_err.write(f"Unable to find {symb} - {ncbi_id_str}\n")
                continue

        response = requests.get(api_base + str(ncbi_id))

        found = False
        while not found:
            response_json = response.json()
            if "error" in response_json and response_json["error"] == 'API rate limit exceeded':
                print(".", end="")
                time.sleep(10)
                found=False
            else:
                found=True
                time.sleep(0.5)

        try:
            gene_summary = response_json["result"][ncbi_id]["summary"]
        except:
            print("error occured in response from ncbi")
            f_err.write("ERROR OCCURED AT {symb} - {ncbi_id_str}: ")
            f_err.write(json.dumps(response_json))
            f_err.write("\n")
            continue
        if not gene_summary:
            gene_summary = "The NCBI Gene database does not have summary information about this gene."
        
        f_out.write(f"{symb}\t{ncbi_id_str}\t{gene_summary}\n")

  0%|          | 0/12610 [00:00<?, ?it/s]

Error occured in finding stuff


In [56]:
unique_metadata_symbol.iloc[5028]

Metadata_JCP2022                   JCP2022_905098
Metadata_broad_sample          ccsbBroad304_05448
Metadata_Name                ORF004605.1_TRC304.1
Metadata_Vector                           pLX_304
Metadata_Transcript                 XLOC_013713_1
Metadata_Symbol                       XLOC_013713
Metadata_NCBI_Gene_ID                 XLOC_013713
Metadata_Taxon_ID                            9606
Metadata_Gene_Description             XLOC_013713
Metadata_Prot_Match                           NaN
Metadata_Insert_Length                      663.0
Metadata_pert_type                            trt
Name: 5096, dtype: object

In [49]:
unique_metadata_symbol.loc[unique_metadata_symbol["Metadata_Symbol"] == "NDUFAF8"]

,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol,Metadata_NCBI_Gene_ID,Metadata_Taxon_ID,Metadata_Gene_Description,Metadata_Prot_Match,Metadata_Insert_Length,Metadata_pert_type
5028,JCP2022_905030,ccsbBroad304_05377,ORF008728.1_TRC304.1,pLX_304,NM_001086521.2,NDUFAF8,284184,9606,NADH:ubiquinone oxidoreductase complex assembl...,100.0,222.0,trt


In [ ]:
# TRICK FOR EMBEDDINGS (IF BAD)
## ADD AN INSTRUCTION
## e.g. "The following is a math question: [equation / question]
## OTHERWISE IT WILL JUST TREAT IT AS SYNTACTIC MEANING

In [72]:
summaries1 = pd.read_csv("/mnt/lab_data2/ryanzhao/gene_drug_graph/summary_data.tsv", sep="\t", names=["symbol", "id", "summary"])
summaries2 = pd.read_csv("/mnt/lab_data2/ryanzhao/gene_drug_graph/summary_data2.tsv", sep="\t", names=["symbol", "id", "summary"])

In [73]:
summaries = pd.concat((summaries1, summaries2), axis=0).reset_index(drop=True)
summaries = summaries.drop_duplicates()

In [74]:
summaries

,symbol,id,summary
0,NAT1,9,This gene is one of two arylamine N-acetyltran...
1,AANAT,15,The protein encoded by this gene belongs to th...
2,ABAT,18,4-aminobutyrate aminotransferase (ABAT) is res...
3,ACADVL,37,The protein encoded by this gene is targeted t...
4,ASIC1,41,This gene encodes a member of the acid-sensing...
...,...,...,...
12479,MME,4311,The protein encoded by this gene is a type II ...
12480,S1PR2,9294,This gene encodes a member of the G protein-co...
12481,HPGDS,27306,Prostaglandin-D synthase is a sigma class glut...
12482,HRH4,59340,Histamine is a ubiquitous messenger molecule r...


In [84]:
client = OpenAI()

In [85]:
output_file = "/mnt/lab_data2/ryanzhao/gene_drug_graph/summary_embeddings.tsv"
total_tokens = 0
rate_limit_sleep_time = 5  # Time to sleep in seconds when rate-limited (adjust based on your plan)

def get_embedding(row):
    """Function to call the OpenAI embeddings API and return relevant data."""
    text = row["symbol"] + ": " + row["summary"]
    
    retries = 5  # Number of times to retry on rate limit
    while retries > 0:
        try:
            # Call OpenAI embeddings API
            response = client.embeddings.create(
                input=text,
                model="text-embedding-3-small"
            )

            usage = response.usage.total_tokens
            embedding = response.data[0].embedding

            # Collect information for writing
            symbol = row["symbol"]
            id = row["id"]
            embedding_json = json.dumps(embedding)
            
            return symbol, id, embedding_json, usage
        
        except Exception as e:
            # Log any other error and stop retrying
            print(f"Error: {e}")
            print(f"Sleeping for {rate_limit_sleep_time} seconds...")
            time.sleep(rate_limit_sleep_time)
            retries -= 1
    
    return None  # Return None if the request failed after retries

# Using ThreadPoolExecutor for parallel API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Write header for the output file
    with open(output_file, "w") as f_out:
        futures = []
        
        # Loop over rows and submit tasks
        for idx, row in summaries.iterrows():
            futures.append(executor.submit(get_embedding, row))
        
        # Process completed futures as they finish
        for future in tqdm(concurrent.futures.as_completed(futures)):
            result = future.result()
            if result:
                symbol, id, embedding_json, usage = result
                
                # Write result to the output file
                f_out.write(f"{symbol}\t{id}\t{embedding_json}\n")
                
                # Update total token usage
                total_tokens += usage

print(f"Total tokens used: {total_tokens}")

0it [00:00, ?it/s]

Total tokens used: 1265134


In [96]:
error_file = "/mnt/lab_data2/ryanzhao/gene_drug_graph/summary_api2.err"

output_file_edgecases = "/mnt/lab_data2/ryanzhao/gene_drug_graph/summary_embeddings_edgecases.tsv"

edgecase_tokens = 0

with open(error_file, "r") as f, open(output_file_edgecases, "w") as f_out:
    for line in tqdm(f):
        line = line.strip()
        gene_symbol = line.split(" ")[3]
        gene_id = line.split(" ")[5]
        # print(gene_symbol, gene_id)

        if gene_symbol == "nan":
            print("skipping nan")
            continue

        text = gene_symbol
        response = client.embeddings.create(
            input=text,
            model="text-embedding-3-small"
        )

        usage = response.usage.total_tokens
        embedding = response.data[0].embedding

        edgecase_tokens += usage

        embedding_json = json.dumps(embedding)

        f_out.write(f"{gene_symbol}\t{gene_id}\t{embedding_json}\n")

print(f"Total edgecase tokens used: {edgecase_tokens}")

0it [00:00, ?it/s]

skipping nan
Total edgecase tokens used: 799
